In [53]:
import json
from datetime import date, datetime, timedelta
import time
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import glob, os
import plotly as py
import plotly.graph_objects as go
import ipywidgets as widgets
import numpy as np

py.offline.init_notebook_mode(connected=True)

msgs_in = []
msgs_out = []
dates = []

os.chdir("") #insert folder dir
for file in glob.glob("*.json"):
    #print(file)
    
    filename = os.getcwd()+"//"+file
    cur_msgs_in, cur_msgs_out, cur_dates = parse_file(filename)
    print(cur_dates[0])
    print(cur_dates[-1])
    if(len(dates)!=0): #if data not empty 
       # print('data not empty')
        delta = dates[-1]-cur_dates[0]
        temp_dates = dates
        temp_msgs_in = msgs_in
        temp_msgs_out = msgs_out
    
        if(delta.days>1):
            #print('missing days')
            for i in range(delta.days):
                temp_dates.append(temp_dates[-1]+timedelta(days=1))
                temp_msgs_in.append(0)
                temp_msgs_out.append(0)
        else:
            #print('overlapping days')
            if(delta.days==0):
                temp_in = cur_msgs_in[-1]
                temp_out = cur_msgs_out[-1]

                
                temp_msgs_in[0]+=temp_in
                temp_msgs_out[0]+=temp_out
                del cur_dates[-1]
                del cur_msgs_in[-1]
                del cur_msgs_out[-1]
            
        msgs_in=temp_msgs_in+cur_msgs_in
        msgs_out=temp_msgs_out+cur_msgs_out
        dates=temp_dates+cur_dates
    else:
        #print('data empty')
        msgs_in = cur_msgs_in
        msgs_out = cur_msgs_out
        dates = cur_dates
    #print(dates[0])
    #print(dates[-1])     
    print('----')
    #print(temp_msgs_in)
    #print(temp_msgs_out)
    #print(temp_dates)
print(dates[0])
print(dates[-1])

2019-10-26
2018-10-12
----
2019-10-26
2018-10-12


In [54]:
def parse_file(filename):
    with open(filename) as f:
        data = json.load(f)
        msgs_out = []
        msgs_in = []
        dates_out = []
        dates_in = []
        dates = []
        counter = 0
        msg_counter = 0
        for message in data['messages']:
            msg_counter+=1
            photos = False
            incoming = False
            cur_date = date.fromtimestamp(message['timestamp_ms'] / 1e3)
            #print(cur_date)
            #print(counter)

            if 'photos' in message:
                photos = True
            if message['sender_name'] != "Kuba Makowiecki":
                incoming = True
            if cur_date in dates: #if date is already present!
                #print('date is present')
                if (photos):
                    if (incoming):
                        msgs_in[counter]+= len(message['photos'])
                        msgs_out[counter]+= 0
                    else:
                        msgs_out[counter]+= len(message['photos'])
                        msgs_in[counter]+= 0
                else:
                    if (incoming):
                        msgs_in[counter]+= 1
                        msgs_out[counter]+= 0
                    else:
                        msgs_out[counter]+= 1
                        msgs_in[counter]+= 0
            else: #if date is NOT present

                if(len(dates)!=0): #if dates is not empty (after the first loop)
                    #print('date is not present')
                    delta = dates[-1]-cur_date  #calculate difference between current date and last date in list
                    #print(delta.days)
                    #print(delta)
                    #print(dates[-1])
                    if (delta.days>0): #if there is a difference
                        for i in range(delta.days):                 #for the number of missing days do:
                            dates.append(dates[-1]-timedelta(days=1))      #add missing dates and add 0 to both msgs
                            counter+=1
                            #print('adding missing date')
                            if (dates[-1]==cur_date): #if last date is current date
                                #print('last date equals current')
                                if (photos):
                                    if (incoming):
                                        msgs_in.append(len(message['photos']))
                                        msgs_out.append(0)
                                    else:
                                        msgs_out.append(len(message['photos']))
                                        msgs_in.append(0)
                                else:
                                    if (incoming):
                                        msgs_in.append(1)
                                        msgs_out.append(0)
                                    else:
                                        msgs_out.append(1)
                                        msgs_in.append(0)
                            else:
                                msgs_in.append(0)
                                msgs_out.append(0)
                else: # first loop (dates are empty)
                    dates.append(cur_date)
                    if (photos):
                        if (incoming):
                            if(msgs_in[counter]):
                                msgs_in[counter]+= len(message['photos'])
                            else:
                                msgs_in.append(len(message['photos']))
                            if(msgs_out[counter]):
                                msgs_out[counter]+= 0
                            else:
                                msgs_out.append(0)
                        else:
                            if(msgs_out[counter]):
                                msgs_out[counter]+= len(message['photos'])
                            else:
                                msgs_out.append(len(message['photos']))
                            if (msgs_in[counter]):
                                msgs_in[counter]+= 0
                            else:
                                msgs_in[counter].append(0)
                    else:
                        if (incoming):
                            msgs_in.append(1)
                            msgs_out.append(0)    
                        else:
                            msgs_out.append(1)
                            msgs_in.append(0)
    return msgs_in,msgs_out,dates
                    #remember to reverse order

In [55]:
print((dates[0]-dates[-1]).days)
print(len(msgs_out))
print(len(msgs_in))
print(len(dates))

379
380
380
380


In [56]:
df=pd.DataFrame({'x':dates, 'y1':msgs_in, 'y2':msgs_out})
#df_sns=pd.DataFrame({msgs_in, msgs_out, dates})

In [57]:
#plt.figure(figsize=(30, 30))
#plt.plot('x','y1', data=df, color='blue')
#plt.plot('x','y2', data=df, color='green')

In [58]:
layout = go.Layout(
title='example',
    yaxis=dict(
        title='number of messages'
    ),
    xaxis=dict(
        title='date'
    )
)

trace1 = go.Scatter(
    x=dates,
    y=msgs_in,
    mode='lines',
    name='msgs_in',
    line=dict(
        shape='spline'
    )
)

trace2 = go.Scatter(
    x=dates,
    y=msgs_out,
    mode='lines',
    name='msgs_out',
    line=dict(
        shape='spline'
    )
)

fig = go.Figure([trace1, trace2], layout=layout)

py.offline.iplot(fig)